<a href="https://colab.research.google.com/github/megmarv/PsychoAI-/blob/Image-Retrieval/FaceExtraction_customModel_01_updated1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import cv2
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras.backend as K

In [15]:
# Directory for face images
output_folder = "extracted_faces"
os.makedirs(output_folder, exist_ok=True)

In [16]:
# Create a function to detect and save faces
def detect_and_save_faces(image_path):
    # Initialize dlib face detector
    detector = dlib.get_frontal_face_detector()
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    face_images = []
    for i, face in enumerate(faces):
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        face_img = gray[y:y+h, x:x+w]
        face_resized = cv2.resize(face_img, (100, 100))
        face_path = os.path.join(output_folder, f"face_{i}.jpg")
        cv2.imwrite(face_path, face_resized)
        face_images.append(face_resized)

    return face_images


In [17]:
# Prepare dataset for training and validation
def prepare_data():
    face_images = []
    labels = []

    # Loop through all files in the directory
    for filename in os.listdir(output_folder):
        if filename.endswith(('.jpg', '.png', '.jpeg')):  # Check if it's an image file
            face_path = os.path.join(output_folder, filename)
            face_img = cv2.imread(face_path, cv2.IMREAD_GRAYSCALE)
            face_resized = cv2.resize(face_img, (100, 100))  # Resize to model input size
            face_resized = np.expand_dims(face_resized, axis=-1)  # Add channel dimension (grayscale)

            face_images.append(face_resized)
            # Here, assign labels based on your use case; for now, we'll assume a label 0
            label = 0  # Update with appropriate label for your case
            labels.append(label)

    # Convert to numpy arrays
    face_images = np.array(face_images)
    labels = np.array(labels)

    # Split into training and validation sets
    train_data, val_data, train_labels, val_labels = train_test_split(face_images, labels, test_size=0.2, random_state=42)

    # Normalize the images
    train_data = train_data / 255.0
    val_data = val_data / 255.0

    # Convert labels to one-hot encoding
    train_labels = to_categorical(train_labels, num_classes=2)  # Assuming two classes
    val_labels = to_categorical(val_labels, num_classes=2)

    return train_data, val_data, train_labels, val_labels


In [18]:
# F1 score metric using TensorFlow
def f1_score_metric(y_true, y_pred):
    # Convert predictions and true labels to binary format (0 or 1)
    y_true = K.argmax(y_true, axis=-1)
    y_pred = K.argmax(y_pred, axis=-1)

    # Calculate precision, recall, and F1 score using TensorFlow operations
    true_positives = K.sum(K.cast(K.equal(y_true, 1) & K.equal(y_pred, 1), K.floatx()))
    false_positives = K.sum(K.cast(K.equal(y_true, 0) & K.equal(y_pred, 1), K.floatx()))
    false_negatives = K.sum(K.cast(K.equal(y_true, 1) & K.equal(y_pred, 0), K.floatx()))

    precision = true_positives / (true_positives + false_positives + K.epsilon())
    recall = true_positives / (true_positives + false_negatives + K.epsilon())
    f1_score = 2 * (precision * recall) / (precision + recall + K.epsilon())

    return f1_score

In [19]:
# Build CNN model
def build_face_recognition_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)),  # Grayscale images have 1 channel
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(2, activation='softmax')  # Assuming binary classification (2 classes)
    ])

    # Compile the model with F1 score as a custom metric
    model.compile(
        optimizer=Adam(),
        loss='categorical_crossentropy',
        metrics=['accuracy', f1_score_metric]  # Add F1 score metric
    )

    return model

if __name__ == "__main__":
    # Specify the directory for group photos
    group_photos_dir = "/content/drive/MyDrive/Group-Photos"

    # Detect and save faces from the group photos
    for filename in os.listdir(group_photos_dir):
        if filename.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(group_photos_dir, filename)
            detect_and_save_faces(image_path)

    # Prepare the dataset for training and validation
    train_data, val_data, train_labels, val_labels = prepare_data()

    # Build and compile the model
    model = build_face_recognition_model()

    # Train the model
    model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=10)

    # Save the trained model
    model.save("face_recognition_model.h5")
    print("Model trained and saved successfully.")


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 1.0000 - f1_score_metric: 0.0000e+00 - loss: 0.6556 - val_accuracy: 1.0000 - val_f1_score_metric: 0.0000e+00 - val_loss: 8.6985e-04
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step - accuracy: 1.0000 - f1_score_metric: 0.0000e+00 - loss: 0.0187 - val_accuracy: 1.0000 - val_f1_score_metric: 0.0000e+00 - val_loss: 3.5763e-07
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step - accuracy: 1.0000 - f1_score_metric: 0.0000e+00 - loss: 2.8116e-04 - val_accuracy: 1.0000 - val_f1_score_metric: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step - accuracy: 1.0000 - f1_score_metric: 0.0000e+00 - loss: 3.6359e-06 - val_accuracy: 1.0000 - val_f1_score_metric: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step - accuracy: 1.0000 - f1_score_metric: 0.0000e+00 - loss: 5.9605e-08 - val_accuracy: 1.0000 - val_f1_score_metric: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 6/10
1/1 ━━━━━━

Model trained and saved successfully.
